# Mise à jour de la BDD de NFT
Source: [binance.com](https://www.binance.com/en/nft/market?currency=&mediaType=&tradeType=&amountFrom=&amountTo=&categorys=&keyword=&page=1&rows=16&productIds=&order=list_time%40-1)

In [1]:
import os
import time
import requests
import numpy as np
from utils import *
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

## 1. Ouverture du marketplace

- Paramétrage du navigateur distant
- Chargement de la page d'accueil
- Activation des cookies 

In [2]:
%%time
start_url = 'https://www.binance.com/en/nft/market?currency=&mediaType=&tradeType=&amountFrom=&amountTo=&categorys=&keyword=&page=1&rows=16&productIds=&order=list_time%40-1'
browser = webdriver.Remote("http://selenium:4444/wd/hub", DesiredCapabilities.FIREFOX)
browser.get(start_url)
cookies = browser.find_element_by_xpath("//button[contains(text(),'Accept')]")
cookies.click()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/magics/execution.py", line 1324, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 2, in <module>
  File "/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 157, in __init__
    self.start_session(capabilities, browser_profile)
  File "/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 252, in start_session
    response = self.execute(Command.NEW_SESSION, parameters)
  File "/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/selenium/webdriver/remote/errorhandler.py", line 242, in check_response
    

TypeError: object of type 'NoneType' has no len()

## 2. Défilement des pages sur le marketplace
 - Choix du nombre de nft à récupérer
 - Chargement des pages contenant les nfts à récuperer 

In [ ]:
%%time

MAX_NFT_NB = 16#select_number()
CURRENT_PAGE_NUMBER = int(browser.current_url.split("page=")[-1].split("&")[0])
NFT_NUMBER_PER_PAGE = int(browser.current_url.split("rows=")[-1].split("&")[0])
if MAX_NFT_NB/NFT_NUMBER_PER_PAGE == MAX_NFT_NB//NFT_NUMBER_PER_PAGE:
  scroll_down = np.arange((MAX_NFT_NB//NFT_NUMBER_PER_PAGE)-1)
else:
  scroll_down = np.arange(MAX_NFT_NB//NFT_NUMBER_PER_PAGE)
for i in tqdm(scroll_down, postfix=' Scrolling down on the main page'):
  while CURRENT_PAGE_NUMBER == (browser.current_url.split("page=")[-1]).split("&")[0]:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  CURRENT_PAGE_NUMBER = (browser.current_url.split("page=")[-1]).split("&")[0]

## 3. Récupération des liens des pages détaillées
- Détection des boutons donnant accès aux infos détaillées

In [ ]:
%%time

detailed_pages = []
pbar = tqdm(total = MAX_NFT_NB, postfix=" Searching of NFT detailed pages")
BAR_LEVEL = 0
while len(detailed_pages) < MAX_NFT_NB:
  browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  detailed_pages = browser.find_elements_by_xpath('//button[contains(text(),"BSC")]')
  pbar.update(max(0,len(detailed_pages)-BAR_LEVEL))
  BAR_LEVEL = len(detailed_pages)
pbar.close()

## 4. Parcours des pages détaillés
- Récupération des données utiles sur les nft
- Nettoyage des chaînes de caractères
- Stockage des dictionnaires de données
- Fermeture du navigateur distant

In [ ]:
%%time
success = 0
failure = 0
total = len(detailed_pages)

for page in tqdm(detailed_pages, postfix=" Scraping of detailed NFT pages"):
  page.click()
  browser.switch_to.window(browser.window_handles[-1])
  res= parse_nft(browser)
  if res.status_code == 201:
        success += 1
  else:
        failure += 1
  browser.close()
  browser.switch_to.window(browser.window_handles[-1])

print(f"Success={success}/{total}; Failure={failure}/{total} - {res.status_code}")
print(f"NFT collection shape: {requests.get('http://api:8000/nfts/').text.count('_id')}")
browser.quit()

## 5. Automatisation du scraping de nfts
- Lancement du processus toutes les 5h

In [1]:
#%%writefile nft.py

from utils import *

WAIT = 3*60 # 5h = 18000s
MAX_NFT_NB = 16
NEXT_START = datetime.now()+timedelta(seconds=WAIT)
start_url = 'https://www.binance.com/en/nft/market?currency=&mediaType=&tradeType=&amountFrom=&amountTo=&categorys=&keyword=&page=1&rows=16&productIds=&order=list_time%40-1'

while True:
    browser = open_binance(start_url) # ouverture du marketplace
    scroll_down(browser, MAX_NFT_NB) # scroll down sur marketplace
    detailed_pages = find_detailed_nft_pages(browser, MAX_NFT_NB) # recherche de pages détaillées
    parse_detailed_nft_pages(browser, detailed_pages) # scraping des pages détaillées
    NEXT_START = wait_next_scraping(NEXT_START, WAIT) # attente avant prochain scraping 

Writing nft.py


## 6. Automatisation du scraping de cryptos
- Lancement du processus toutes les 15 minutes

In [1]:
#%%writefile crypto.py

from utils import *

WAIT = 3*60 # 15 min = 900s
NEXT_START = datetime.now()+timedelta(seconds=WAIT)
start_url = 'https://www.binance.com/fr/markets'

while True:
    browser = open_binance(start_url, False) # ouverture de la page principale des cryptos
    nb_pages = find_crypto_pages(browser) # recherche de pages additionnelles
    parse_crypto_pages(browser, nb_pages) # scraping des pages de cryptos
    NEXT_START = wait_next_scraping(NEXT_START, WAIT) # attente avant prochain scraping

2021-09-30 16:46:46: New scraping initiated
2021-09-30 16:47:01: Binance page correctly opened
2021-09-30 16:47:01: Searching of crypto pages


2021-09-30 16:47:02 Scraping of crypto pages: 100%|██████████| 19/19 [00:21<00:00,  1.15s/it]


[{'Acronym': 'BTC', 'Name': 'Bitcoin', 'Price': 37075.08, 'Date': '2021-09-30 16:47:24.206061', 'Currency': '€'}, {'Acronym': 'ETH', 'Name': 'Ethereum', 'Price': 2567.92, 'Date': '2021-09-30 16:47:24.206061', 'Currency': '€'}, {'Acronym': 'USDT', 'Name': 'TetherUS', 'Price': 0.86, 'Date': '2021-09-30 16:47:24.206061', 'Currency': '€'}, {'Acronym': 'ADA', 'Name': 'Cardano', 'Price': 1.8, 'Date': '2021-09-30 16:47:24.206061', 'Currency': '€'}, {'Acronym': 'BNB', 'Name': 'BNB', 'Price': 327.92, 'Date': '2021-09-30 16:47:24.206061', 'Currency': '€'}, {'Acronym': 'XRP', 'Name': 'Ripple', 'Price': 0.81, 'Date': '2021-09-30 16:47:24.206061', 'Currency': '€'}, {'Acronym': 'SOL', 'Name': 'Solana', 'Price': 118.24, 'Date': '2021-09-30 16:47:24.206061', 'Currency': '€'}, {'Acronym': 'USDC', 'Name': 'USD Coin', 'Price': 0.86, 'Date': '2021-09-30 16:47:24.206061', 'Currency': '€'}, {'Acronym': 'DOT', 'Name': 'Polkadot', 'Price': 24.11, 'Date': '2021-09-30 16:47:24.206061', 'Currency': '€'}, {'Acron

2021-09-30 16:47:25 Wait for 140 seconds


KeyboardInterrupt: 